# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.columns

Index(['Student ID', 'student_name', 'gender', 'grade', 'school_name',
       'reading_score', 'math_score', 'School ID', 'type', 'size', 'budget'],
      dtype='object')

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [77]:
Total_Schools = school_data_complete["School ID"].unique().sum()
Total_Students = school_data_complete["Student ID"].unique().sum()
Total_Budget = school_data_complete["budget"].sum()
Average_Math = school_data_complete["math_score"].mean()
Average_Read = school_data_complete["reading_score"].mean()

Total_Avg_Grade = (Average_Math + Average_Read) / 2

Pass_Math_Per = (len(school_data_complete[school_data_complete["math_score"] >= 70]) / Total_Students)
Pass_Read_Per = (len(school_data_complete[school_data_complete["reading_score"] >= 70]) / Total_Students)

Dist_df = pd.DataFrame({'Total Schools':Total_Schools, 'Total Students':Total_Students, 'Total Budget':Total_Budget, \
                         'Avg Math Score':Average_Math, 'Avg Read Score':Average_Read, 'Total Avg Score':Total_Avg_Grade, \
                        'Pass Math Per':Pass_Math_Per, 'Pass Read Per':Pass_Read_Per}, index=[0])

Dist_df



,Total Schools,Total Students,Total Budget,Avg Math Score,Avg Read Score,Total Avg Score,Pass Math Per,Pass Read Per
0,105,767124865,82932329558,78.985371,81.87784,80.431606,0.000038,0.000044


In [78]:
#School Summary here. Also for the record, unless there is something I am missing...,Excel is faster at this.

school_data_complete.columns

School_List = school_data_complete["school_name"].unique()
School_Summary = pd.DataFrame({"school_name":School_List})

School_type = school_data_complete.loc[:,["school_name", "type"]]
School_type = School_type.groupby(['school_name'])['type'].unique()
df1 = pd.DataFrame([School_type], index=['type']).T

new_summary1 = pd.merge(School_Summary, df1 , on="school_name")

Student_Count = school_data_complete.loc[:,["school_name", "Student ID"]]
Student_Count = Student_Count.groupby(['school_name'])['Student ID'].count()
df2 = pd.DataFrame([Student_Count], index=['Student ID']).T

new_summary2 = pd.merge(new_summary1, df2, on="school_name")
new_summary2 = new_summary2.rename(index=str, columns={"Student ID": "Total Students"})

School_Budget = school_data_complete.loc[:,["school_name", "budget"]]
School_Budget = School_Budget.groupby(['school_name'])['budget'].unique()
df3 = pd.DataFrame([School_Budget], index=['Budget']).T

new_summary3 = pd.merge(new_summary2, df3, on="school_name")

new_summary3["Bud Per Stu"] = new_summary3["Budget"] / new_summary3["Total Students"]

new_summary3

Avg_Math = school_data_complete.loc[:,["school_name", "math_score"]]
Avg_Math = Avg_Math.groupby(['school_name'])['math_score'].mean()
df4 = pd.DataFrame([Avg_Math], index=['math_score']).T

new_summary4 = pd.merge(new_summary3, df4, on="school_name")

Avg_Read = school_data_complete.loc[:,["school_name", "reading_score"]]
Avg_Read = Avg_Read.groupby(['school_name'])['reading_score'].mean()
df5 = pd.DataFrame([Avg_Read], index=['reading_score']).T

new_summary5 = pd.merge(new_summary4, df5, on="school_name")

Math_Win = school_data_complete.loc[:,["school_name", "math_score"]]
Math_Win = Math_Win[Math_Win["math_score"] >= 70]
Math_Win = Math_Win.groupby(['school_name'])['math_score'].count()
df6 = pd.DataFrame([Math_Win], index=['math_score']).T

new_summary6 = pd.merge(new_summary5, df6, on="school_name")
new_summary6["% Pass Math"] = new_summary6["math_score_y"] / new_summary6["Total Students"]
new_summary6 = new_summary6.drop(columns=['math_score_y'])
new_summary6 = new_summary6.rename(index=str, columns={"math_score_x": "math_score"})

Read_Win = school_data_complete.loc[:,["school_name", "reading_score"]]
Read_Win = Read_Win[Read_Win["reading_score"] >= 70]
Read_Win = Read_Win.groupby(['school_name'])['reading_score'].count()
df7 = pd.DataFrame([Read_Win], index=['reading_score']).T

final_summary = pd.merge(new_summary6, df7, on="school_name")
final_summary["% Pass Read"] = final_summary["reading_score_y"] / final_summary["Total Students"]
final_summary = final_summary.drop(columns=['reading_score_y'])
final_summary = final_summary.rename(index=str, columns={"reading_score_x": "reading_score"})

final_summary["% Overall Pass"] = (final_summary["% Pass Math"] + final_summary["% Pass Read"]) / 2

final_summary = final_summary.astype({"Budget": int, "Bud Per Stu": int})
final_summary


,school_name,type,Total Students,Budget,Bud Per Stu,math_score,reading_score,% Pass Math,% Pass Read,% Overall Pass
0,Huang High School,[District],2917,1910635,655,76.629414,81.182722,0.656839,0.813164,0.735002
1,Figueroa High School,[District],2949,1884411,639,76.711767,81.158020,0.659885,0.807392,0.733639
2,Shelton High School,[Charter],1761,1056600,600,83.359455,83.725724,0.938671,0.958546,0.948609
3,Hernandez High School,[District],4635,3022020,652,77.289752,80.934412,0.667530,0.808630,0.738080
4,Griffin High School,[Charter],1468,917500,625,83.351499,83.816757,0.933924,0.971390,0.952657
5,Wilson High School,[Charter],2283,1319574,578,83.274201,83.989488,0.938677,0.965396,0.952037
6,Cabrera High School,[Charter],1858,1081356,582,83.061895,83.975780,0.941335,0.970398,0.955867
7,Bailey High School,[District],4976,3124928,628,77.048432,81.033963,0.666801,0.819333,0.743067
8,Holden High School,[Charter],427,248087,581,83.803279,83.814988,0.925059,0.962529,0.943794
9,Pena High School,[Charter],962,585858,609,83.839917,84.044699,0.945946,0.959459,0.952703


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [79]:
Top_performers = final_summary.sort_values(by=["% Overall Pass"], ascending=False)
Top_performers.head(5).set_index('school_name')

,type,Total Students,Budget,Bud Per Stu,math_score,reading_score,% Pass Math,% Pass Read,% Overall Pass
school_name,,,,,,,,,
Cabrera High School,[Charter],1858,1081356,582,83.061895,83.975780,0.941335,0.970398,0.955867
Thomas High School,[Charter],1635,1043130,638,83.418349,83.848930,0.932722,0.973089,0.952905
Pena High School,[Charter],962,585858,609,83.839917,84.044699,0.945946,0.959459,0.952703
Griffin High School,[Charter],1468,917500,625,83.351499,83.816757,0.933924,0.971390,0.952657
Wilson High School,[Charter],2283,1319574,578,83.274201,83.989488,0.938677,0.965396,0.952037


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [80]:
Worst_performers = final_summary.sort_values(by=["% Overall Pass"], ascending=True)
Worst_performers.head(5).set_index('school_name')

,type,Total Students,Budget,Bud Per Stu,math_score,reading_score,% Pass Math,% Pass Read,% Overall Pass
school_name,,,,,,,,,
Rodriguez High School,[District],3999,2547363,637,76.842711,80.744686,0.663666,0.802201,0.732933
Figueroa High School,[District],2949,1884411,639,76.711767,81.158020,0.659885,0.807392,0.733639
Huang High School,[District],2917,1910635,655,76.629414,81.182722,0.656839,0.813164,0.735002
Johnson High School,[District],4761,3094650,650,77.072464,80.966394,0.660576,0.812224,0.736400
Ford High School,[District],2739,1763916,644,77.102592,80.746258,0.683096,0.792990,0.738043


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [81]:
Grades = school_data_complete.loc[:,["school_name", "grade","math_score"]]

ninth = Grades[Grades["grade"] == "9th"]
ninth_sum = ninth.groupby(['school_name'])['math_score'].sum()
ninth_sum_df = pd.DataFrame([ninth_sum], index=['math_sum']).T

ninth_count = ninth.groupby(['school_name'])['math_score'].count()
ninth_count_df = pd.DataFrame([ninth_count], index=['grade_count']).T

ninth_final = pd.merge(ninth_sum_df, ninth_count_df, on="school_name")
ninth_final["9th"] = ninth_final["math_sum"] / ninth_final["grade_count"]
ninth_final = ninth_final.drop(columns=['math_sum','grade_count'])

tenth = Grades[Grades["grade"] == "10th"]
tenth_sum = tenth.groupby(['school_name'])['math_score'].sum()
tenth_sum_df = pd.DataFrame([tenth_sum], index=['math_sum']).T

tenth_count = tenth.groupby(['school_name'])['math_score'].count()
tenth_count_df = pd.DataFrame([tenth_count], index=['grade_count']).T

tenth_final = pd.merge(tenth_sum_df, tenth_count_df, on="school_name")
tenth_final["10th"] = tenth_final["math_sum"] / tenth_final["grade_count"]
tenth_final = tenth_final.drop(columns=['math_sum','grade_count'])

eleventh = Grades[Grades["grade"] == "11th"]
eleventh_sum = eleventh.groupby(['school_name'])['math_score'].sum()
eleventh_sum_df = pd.DataFrame([eleventh_sum], index=['math_sum']).T

eleventh_count = eleventh.groupby(['school_name'])['math_score'].count()
eleventh_count_df = pd.DataFrame([eleventh_count], index=['grade_count']).T

eleventh_final = pd.merge(eleventh_sum_df, eleventh_count_df, on="school_name")
eleventh_final["11th"] = eleventh_final["math_sum"] / eleventh_final["grade_count"]
eleventh_final = eleventh_final.drop(columns=['math_sum','grade_count'])

twelfth = Grades[Grades["grade"] == "12th"]
twelfth_sum = twelfth.groupby(['school_name'])['math_score'].sum()
twelfth_sum_df = pd.DataFrame([twelfth_sum], index=['math_sum']).T

twelfth_count = eleventh.groupby(['school_name'])['math_score'].count()
twelfth_count_df = pd.DataFrame([twelfth_count], index=['grade_count']).T

twelfth_final = pd.merge(twelfth_sum_df, twelfth_count_df, on="school_name")
twelfth_final["12th"] = twelfth_final["math_sum"] / twelfth_final["grade_count"]
twelfth_final = twelfth_final.drop(columns=['math_sum','grade_count'])

df1 = pd.merge(ninth_final, tenth_final, on="school_name")
df2 = pd.merge(df1,eleventh_final, on="school_name")
df3 = pd.merge(df2,twelfth_final, on="school_name")

df3

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,62.856914
Cabrera High School,83.094697,83.154506,82.765560,66.000000
Figueroa High School,76.403037,76.539974,76.884344,67.575458
Ford High School,77.361345,77.672316,76.918058,62.308042
Griffin High School,82.044010,84.229064,83.842105,67.423823
Hernandez High School,77.438495,77.337408,77.136029,66.545037
Holden High School,83.787402,83.429825,85.000000,66.766990
Huang High School,77.027251,75.908735,76.446602,62.658807
Johnson High School,77.187857,76.691117,77.491653,60.053422


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [82]:
Grades = school_data_complete.loc[:,["school_name", "grade","reading_score"]]

ninth = Grades[Grades["grade"] == "9th"]
ninth_sum = ninth.groupby(['school_name'])['reading_score'].sum()
ninth_sum_df = pd.DataFrame([ninth_sum], index=['reading_sum']).T

ninth_count = ninth.groupby(['school_name'])['reading_score'].count()
ninth_count_df = pd.DataFrame([ninth_count], index=['grade_count']).T

ninth_final = pd.merge(ninth_sum_df, ninth_count_df, on="school_name")
ninth_final["9th"] = ninth_final["reading_sum"] / ninth_final["grade_count"]
ninth_final = ninth_final.drop(columns=['reading_sum','grade_count'])

tenth = Grades[Grades["grade"] == "10th"]
tenth_sum = tenth.groupby(['school_name'])['reading_score'].sum()
tenth_sum_df = pd.DataFrame([tenth_sum], index=['reading_sum']).T

tenth_count = tenth.groupby(['school_name'])['reading_score'].count()
tenth_count_df = pd.DataFrame([tenth_count], index=['grade_count']).T

tenth_final = pd.merge(tenth_sum_df, tenth_count_df, on="school_name")
tenth_final["10th"] = tenth_final["reading_sum"] / tenth_final["grade_count"]
tenth_final = tenth_final.drop(columns=['reading_sum','grade_count'])

eleventh = Grades[Grades["grade"] == "11th"]
eleventh_sum = eleventh.groupby(['school_name'])['reading_score'].sum()
eleventh_sum_df = pd.DataFrame([eleventh_sum], index=['reading_sum']).T

eleventh_count = eleventh.groupby(['school_name'])['reading_score'].count()
eleventh_count_df = pd.DataFrame([eleventh_count], index=['grade_count']).T

eleventh_final = pd.merge(eleventh_sum_df, eleventh_count_df, on="school_name")
eleventh_final["11th"] = eleventh_final["reading_sum"] / eleventh_final["grade_count"]
eleventh_final = eleventh_final.drop(columns=['reading_sum','grade_count'])

twelfth = Grades[Grades["grade"] == "12th"]
twelfth_sum = twelfth.groupby(['school_name'])['reading_score'].sum()
twelfth_sum_df = pd.DataFrame([twelfth_sum], index=['reading_sum']).T

twelfth_count = eleventh.groupby(['school_name'])['reading_score'].count()
twelfth_count_df = pd.DataFrame([twelfth_count], index=['grade_count']).T

twelfth_final = pd.merge(twelfth_sum_df, twelfth_count_df, on="school_name")
twelfth_final["12th"] = twelfth_final["reading_sum"] / twelfth_final["grade_count"]
twelfth_final = twelfth_final.drop(columns=['reading_sum','grade_count'])

df1 = pd.merge(ninth_final, tenth_final, on="school_name")
df2 = pd.merge(df1,eleventh_final, on="school_name")
df3 = pd.merge(df2,twelfth_final, on="school_name")

df3

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,66.489209
Cabrera High School,83.676136,84.253219,83.788382,66.800830
Figueroa High School,81.198598,81.408912,80.640339,71.283498
Ford High School,80.632653,81.262712,80.403642,65.974203
Griffin High School,83.369193,83.706897,84.288089,67.955679
Hernandez High School,80.866860,80.660147,81.396140,69.709559
Holden High School,83.677165,83.324561,83.815534,68.252427
Huang High School,81.290284,81.512386,81.417476,65.158114
Johnson High School,81.260714,80.773431,80.616027,63.463272


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [83]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [84]:
spending_summary = final_summary.drop(columns=['school_name','type','Total Students','Budget'])

spending_summary["Spending Ranges"] = pd.cut(spending_summary["Bud Per Stu"], spending_bins, labels=group_names)

spending_summary = spending_summary.drop(columns=['Bud Per Stu'])

spending_summary = spending_summary.groupby("Spending Ranges")

spending_summary.max()

,math_score,reading_score,% Pass Math,% Pass Read,% Overall Pass
Spending Ranges,,,,,
<$585,83.803279,83.989488,0.941335,0.970398,0.955867
$585-615,83.839917,84.044699,0.945946,0.959459,0.952703
$615-645,83.418349,83.848930,0.933924,0.973089,0.952905
$645-675,77.289752,81.182722,0.667530,0.813164,0.738080


## Scores by School Size

* Perform the same operations as above, based on school size.

In [85]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names1 = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [86]:
size_summary = final_summary.drop(columns=['school_name','type','Bud Per Stu','Budget'])
size_summary
size_summary["Size Ranges"] = pd.cut(size_summary["Total Students"], size_bins, labels=group_names1)

size_summary = size_summary.drop(columns=['Total Students'])

size_summary = size_summary.groupby("Size Ranges")

size_summary.max()

,math_score,reading_score,% Pass Math,% Pass Read,% Overall Pass
Size Ranges,,,,,
Small (<1000),83.839917,84.044699,0.945946,0.962529,0.952703
Medium (1000-2000),83.682222,83.975780,0.941335,0.973089,0.955867
Large (2000-5000),83.274201,83.989488,0.938677,0.965396,0.952037


## Scores by School Type

* Perform the same operations as above, based on school type.

In [87]:
type_summary = final_summary.drop(columns=['school_name','Total Students','Bud Per Stu','Budget'])
type_summary = type_summary.astype({"type": str})

type_summary = type_summary.groupby("type")

type_summary.max()

,math_score,reading_score,% Pass Math,% Pass Read,% Overall Pass
type,,,,,
['Charter'],83.839917,84.044699,0.945946,0.973089,0.955867
['District'],77.289752,81.182722,0.683096,0.819333,0.743067
